### Prompt:
You are an analyst for a fastfood restaurant. The company has tasked you to analyze their past transactions. Each row represents a single transaction/order from one of their ten (10) branches. The order is recorded as a __`string`__ which is formatted as: 

"`[quantity1][item_code1], [quantity2][item_code2], [quantity3][item_code3]`"

However, because of some technical error, the __number of combo meals purchased__ as well as the __total order value__ was lost, so we'll need to restore that.

### You are tasked to come up with the following:
1. Given the order column, count how many of each product was purchased
2. Given each product count, get the number of supposed combo meals purchased. <br><i>(You may update the product count when the combo meals are counted)</i>.
3. Calculate the total order value.
4. Determine which branch made the most revenue, and how much?

### The combo meals are detailed below:
1. Family combo: 4 burgers, 4 fries, 4 drinks
2. Big combo: 1 burgers, 1 fries, 1 drinks
3. Snack combo: 1 fries, 1 drinks

In [1]:
import numpy as np
import pandas as pd

resto = pd.read_csv("food_transactions.csv")
resto

,Unnamed: 0,date,branch,order
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink"
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink"
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink"
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink"
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink"
...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink"
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries"
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries"
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink"


In [2]:
import json

jsondf = pd.read_json("price_list.json")
jsondf

,price
Big_combo,169
Fam_combo,599
S_Burger,89
S_Drink,49
S_Fries,69
Snack_combo,99


In [4]:
new_data = resto["order"].str.split(",", expand=True)

new_data[['number_1', 'order_1']] = new_data[0].str.split("S_", expand=True)
new_data[['number_2', 'order_2']] = new_data[1].str.split("S_", expand=True)
new_data[['number_3', 'order_3']] = new_data[2].str.split("S_", expand=True)

new_data

# print(new_data.groupby("order_1").size())
# print(new_data.groupby("order_2").size())
# print(new_data.groupby("order_3").size())

# order_1 = new_data.groupby("order_1").size()
# order_2 = new_data.groupby("order_2").size()
# order_3 = new_data.groupby("order_3").size()

Burgers = order_1["Burger"]
Drinks = order_1["Drink"]+order_2["Drink"]+order_3["Drink"]
Fries = order_1["Fries"]+order_2["Fries"]

print("Burgers: " + str(Burgers))
print("Drinks: " + str(Drinks))
print("Fries: " + str(Fries))

NameError: name 'order_1' is not defined

In [ ]:
# 2. Given each product count, get the number of supposed combo meals purchased.
# (You may update the product count when the combo meals are counted).
# The combo meals are detailed below:
# Family combo: 4 burgers, 4 fries, 4 drinks
# Big combo: 1 burgers, 1 fries, 1 drinks
# Snack combo: 1 fries, 1 drinks

family_combo = len(resto[resto["order"]=="4S_Burger, 4S_Fries, 4S_Drink"])
big_combo = len(resto[resto["order"]=="1S_Burger, 1S_Fries, 1S_Drink"])
snack_combo = len(resto[resto["order"]=="1S_Fries, 1S_Drink"])
print("Family Combo: " + str(family_combo))
print("Big Combo: " + str(big_combo))
print("Snack Combo: " + str(snack_combo))

In [ ]:
# 3. Calculate the total order value.

print("Family Combo: " + str((family_combo) * (jsondf.loc["Fam_combo", "price"])))
print("Big Combo: " + str((big_combo) * (jsondf.loc["Big_combo", "price"])))
print("Snack Combo: " + str((snack_combo) * (jsondf.loc["Snack_combo", "price"])))

fries_indiv = Fries - family_combo*4 - big_combo*1 - snack_combo*1
burger_indiv = Burgers - family_combo*4 - big_combo*1
drinks_indiv = Drinks - family_combo*4 - big_combo*1 - snack_combo*1

print("Fries: " + str(fries_indiv))
print("Burger: " + str(burger_indiv))
print("Drinks: " + str(drinks_indiv))

In [ ]:
# 4. Determine which branch made the most revenue, and how much?

final = resto.merge(new_data, how = "left", left_on = "date", right_on = "date")